In [25]:
import xlrd, xlwt
from xlwt import Workbook

In [26]:
excelPath = ("FullDatabase04082020.xlsx")

In [27]:
workbook = xlrd.open_workbook(excelPath) # read workbook
sheet = workbook.sheet_by_index(0)

In [28]:
firstRow = []
for i in range(sheet.ncols):
    firstRow.append(sheet.cell_value(0, i))

In [29]:
Title, Authors, Journal, DOI = [], [], [], []
for i in range(0, sheet.nrows):
    Title.append(sheet.cell_value(i, 0))
    Authors.append(sheet.cell_value(i, 1))
    Journal.append(sheet.cell_value(i, 5))
    DOI.append(sheet.cell_value(i, 10))

In [30]:
wb = Workbook() # write workbook 
filteredSheet = wb.add_sheet("Filtered Sheet")

In [31]:
# write filtered data
for i in range(len(Authors)):
    filteredSheet.write(i, 0, Title[i])
    filteredSheet.write(i, 1, Authors[i])
    filteredSheet.write(i, 2, Journal[i])
    filteredSheet.write(i, 3, DOI[i])

In [32]:
wb.save("FilteredDatabase.xls")